<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Launch Sites Locations Analysis with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:
- **TASK 1:** Mark all launch sites on a map
- **TASK 2:** Mark the success/failed launches for each site on the map
- **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [7]:
!pip3 install folium
!pip3 install wget
!pip3 install pandas

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9680 sha256=c207ef135f50ba89085511b776cf7bb30300601b46056ec0fba0734482e2bd11
  Stored in directory: c:\users\cash\appdata\local\pip\cache\wheels\01\46\3b\e29ffbe4ebe614ff224bad40fc6a5773a67a163251585a13a9
Successfully built wget


In [9]:
import folium
import wget
import pandas as pd

In [116]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/DV0101EN-3-5-1-Generating-Maps-in-Python-py-v2.0.ipynb)


## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site. 


In [123]:
# Download and read the `spacex_launch_geo.csv`
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [125]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [130]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example, 


In [133]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle. 


Now, let's add a circle for each launch site in data frame `launch_sites`


_TODO:_  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [142]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
# Define coordinates for NASA center (or use average of launch sites)
nasa_coordinate = [28.562302, -80.577356]

# Create DataFrame for launch sites
launch_sites = pd.DataFrame({
    'LaunchSite': ['CCAFS LC-40', 'CCAFS SLC-40', 'KSC LC-39A', 'VAFB SLC-4E'],
    'Lat': [28.562302, 28.563197, 28.573255, 34.632834],
    'Long': [-80.577356, -80.576820, -80.646895, -120.610745]
})

# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# For each launch site, add a Circle object based on its coordinate (Lat, Long) values.
# In addition, add Launch site name as a popup label
for index, row in launch_sites.iterrows():
    coordinate = [row['Lat'], row['Long']]
    
    # Add Circle
    folium.Circle(
        location=coordinate,
        radius=1000,
        color='#000000',
        fill=True
    ).add_child(folium.Popup(row['LaunchSite'])).add_to(site_map)
    
    # Add Marker with text label
    folium.Marker(
        location=coordinate,
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % row['LaunchSite']
        )
    ).add_to(site_map)

# Display the map
site_map

The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:
- Are all launch sites in proximity to the Equator line?
- Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


# Task 2: Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [149]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records. 
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [154]:
marker_cluster = MarkerCluster()


In [156]:
print(launch_sites.columns)

Index(['LaunchSite', 'Lat', 'Long'], dtype='object')


_TODO:_ Create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value


In [169]:
# created dataframe as on previous table was not included 
data = {
    'launchsite': [
        'KSC LC-39A', 'KSC LC-39A', 'KSC LC-39A',
        'CCAFS SLC-40', 'CCAFS SLC-40', 'CCAFS SLC-40', 
        'CCAFS SLC-40', 'CCAFS SLC-40', 'CCAFS SLC-40', 
        'CCAFS SLC-40'
    ],
    'lat': [
        28.573255, 28.573255, 28.573255,
        28.563197, 28.563197, 28.563197, 
        28.563197, 28.563197, 28.563197,
        28.563197
    ],
    'long': [
        -80.646895, -80.646895, -80.646895,
        -80.576820, -80.576820, -80.576820, 
        -80.576820, -80.576820, -80.576820,
        -80.576820
    ],
    'class': [
        1, 1, 1,
        1, 1, 0,
        0, 0, 1,
        0
    ]
}

launch_sites = pd.DataFrame(data)
# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red
# Step 1: Clean column names (in case of hidden spaces or inconsistent casing)
launch_sites.columns = launch_sites.columns.str.strip().str.lower()

launch_sites['marker_color'] = launch_sites['class'].apply(lambda x: 'green' if x == 1 else 'red')

In [171]:
# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


_TODO:_ For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [189]:
site_map = folium.Map(location=[spacex_df['Lat'].mean(), spacex_df['Long'].mean()], zoom_start=6)
# Add marker_cluster to current site_map
# Initialize the map centered around average coordinates
site_map = folium.Map(location=[spacex_df['Lat'].mean(), spacex_df['Long'].mean()], zoom_start=6)

# Create a MarkerCluster object
marker_cluster = MarkerCluster()

# Loop through each row and add a marker to the cluster
for idx, row in spacex_df.iterrows():
    marker = folium.Marker(
        location=[row['Lat'], row['Long']],
        popup=f"Launch Site: {row['Launch Site']}<br>Class: {row['class']}",
        icon=folium.Icon(color=row['marker_color'])
    )
    marker_cluster.add_child(marker)

# Add marker cluster to map
site_map.add_child(marker_cluster)

# Display the map
site_map

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [197]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


You can calculate the distance between two points on the map based on their `Lat` and `Long` values using the following method:


In [201]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

_TODO:_ Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [218]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
launch_sites['Distance_to_Coast_km'] = launch_sites.apply(
    lambda row: calculate_distance(row['lat'], row['long'], coastline_lat, coastline_lon),
    axis=1
)

print(launch_sites[['launchsite', 'lat', 'long', 'Distance_to_Coast_km']])

     launchsite        lat       long  Distance_to_Coast_km
0    KSC LC-39A  28.573255 -80.646895              7.429322
1    KSC LC-39A  28.573255 -80.646895              7.429322
2    KSC LC-39A  28.573255 -80.646895              7.429322
3  CCAFS SLC-40  28.563197 -80.576820              0.509743
4  CCAFS SLC-40  28.563197 -80.576820              0.509743
5  CCAFS SLC-40  28.563197 -80.576820              0.509743
6  CCAFS SLC-40  28.563197 -80.576820              0.509743
7  CCAFS SLC-40  28.563197 -80.576820              0.509743
8  CCAFS SLC-40  28.563197 -80.576820              0.509743
9  CCAFS SLC-40  28.563197 -80.576820              0.509743


_TODO:_ After obtained its coordinate, create a `folium.Marker` to show the distance


In [220]:
site_map = folium.Map(location=[launch_sites['lat'].mean(), launch_sites['long'].mean()], zoom_start=5)

# Add a marker for each launch site showing distance to coastline
for idx, row in launch_sites.iterrows():
    folium.Marker(
        location=[row['lat'], row['long']],
        popup=folium.Popup(f"{row['launchsite']}<br>Distance to Coastline: {row['Distance_to_Coast_km']:.2f} km", max_width=300),
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(site_map)

# Display the map
site_map

_TODO:_ Draw a `PolyLine` between a launch site to the selected coastline point


In [236]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)
closest_railway_lat = 28.5721
closest_railway_lon = -80.5851

for idx, row in launch_sites.iterrows():
    launch_coord = [row['lat'], row['long']]
    coast_coord = [coastline_lat, coastline_lon]
    
    # Create PolyLine object
    lines = folium.PolyLine(locations=[launch_coord, coast_coord], weight=1, color='blue')
    
    # Add the line to the map
    site_map.add_child(lines)

# Display the map
site_map

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


_TODO:_ Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [ ]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site


In [238]:
closest_city_lat = 28.5678
closest_city_lon = -80.5750

closest_railway_lat = 28.5721
closest_railway_lon = -80.5851

for idx, row in launch_sites.iterrows():
    launch_coord = [row['lat'], row['long']]
    city_coord = [closest_city_lat, closest_city_lon]

    # Marker for launch site
    folium.Marker(
        location=launch_coord,
        popup=folium.Popup(f"{row['launchsite']}<br>Distance to Coastline: {row['Distance_to_Coast_km']:.2f} km"),
        icon=folium.Icon(color='blue', icon='rocket')
    ).add_to(site_map)

    # Marker for closest city (or railway/highway)
    folium.Marker(
        location=city_coord,
        popup='Closest City/Railway/Highway',
        icon=folium.Icon(color='green', icon='info-sign')
    ).add_to(site_map)

    # Draw a line between launch site and closest city
    folium.PolyLine(
        locations=[launch_coord, city_coord],
        color='purple',
        weight=2.5,
        opacity=0.7
    ).add_to(site_map)

site_map

After you plot distance lines to the proximities, you can answer the following questions easily:
- Are launch sites in close proximity to railways?
- Are launch sites in close proximity to highways?
- Are launch sites in close proximity to coastline?
- Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


In [242]:
# Example closest highway coordinates
closest_highway_lat = 28.5623
closest_highway_lon = -80.5775

launch_sites['Distance_to_Highway_km'] = launch_sites.apply(
    lambda row: calculate_distance(row['lat'], row['long'], closest_highway_lat, closest_highway_lon),
    axis=1
)

# Example closest city coordinates
closest_city_lat = 28.5383
closest_city_lon = -81.3792  # Orlando, FL for example

launch_sites['Distance_to_City_km'] = launch_sites.apply(
    lambda row: calculate_distance(row['lat'], row['long'], closest_city_lat, closest_city_lon),
    axis=1
)

In [250]:
# Define proximity thresholds
coastline_threshold_km = 20      # e.g., within 20 km is considered close to coast
highway_threshold_km = 10        # e.g., within 10 km is considered close to highway
city_min_distance_km = 30        # e.g., at least 30 km away from city is considered safe

# Proximity summary
summary = {
    'Coastline close to all launch sites?': (launch_sites['Distance_to_Coast_km'] <= coastline_threshold_km).all(),
    'Highway close to all launch sites?': (launch_sites['Distance_to_Highway_km'] <= highway_threshold_km).all(),
    'Launch sites keep safe distance from cities?': (launch_sites['Distance_to_City_km'] >= city_min_distance_km).all()
}

# Print the results
for key, val in summary.items():
    print(f"{key}: {'Yes' if val else 'No'}")


Coastline close to all launch sites?: Yes
Highway close to all launch sites?: Yes
Launch sites keep safe distance from cities?: Yes


In [252]:
print("\n--- Detailed distances for each launch site ---")
print(launch_sites[['launchsite', 'Distance_to_Coast_km', 'Distance_to_Highway_km', 'Distance_to_City_km']])


--- Detailed distances for each launch site ---
     launchsite  Distance_to_Coast_km  Distance_to_Highway_km  \
0    KSC LC-39A              7.429322                6.887690   
1    KSC LC-39A              7.429322                6.887690   
2    KSC LC-39A              7.429322                6.887690   
3  CCAFS SLC-40              0.509743                0.119866   
4  CCAFS SLC-40              0.509743                0.119866   
5  CCAFS SLC-40              0.509743                0.119866   
6  CCAFS SLC-40              0.509743                0.119866   
7  CCAFS SLC-40              0.509743                0.119866   
8  CCAFS SLC-40              0.509743                0.119866   
9  CCAFS SLC-40              0.509743                0.119866   

   Distance_to_City_km  
0            71.650896  
1            71.650896  
2            71.650896  
3            78.444191  
4            78.444191  
5            78.444191  
6            78.444191  
7            78.444191  
8         

In [254]:
import folium
import math

# Create a base map centered around the first launch site
m = folium.Map(location=[launch_sites['lat'].mean(), launch_sites['long'].mean()], zoom_start=6)

# Function to estimate endpoint based on distance and bearing
def destination_point(lat, lon, distance_km, bearing_deg):
    R = 6371  # Earth radius in km
    bearing_rad = math.radians(bearing_deg)
    
    lat1 = math.radians(lat)
    lon1 = math.radians(lon)

    lat2 = math.asin(math.sin(lat1)*math.cos(distance_km/R) +
                     math.cos(lat1)*math.sin(distance_km/R)*math.cos(bearing_rad))

    lon2 = lon1 + math.atan2(math.sin(bearing_rad)*math.sin(distance_km/R)*math.cos(lat1),
                             math.cos(distance_km/R)-math.sin(lat1)*math.sin(lat2))

    return math.degrees(lat2), math.degrees(lon2)

# Bearings (in degrees): Coast = 90° (East), Highway = 0° (North), City = 270° (West)
bearings = {
    'Coast': 90,
    'Highway': 0,
    'City': 270
}

# Draw launch sites and connecting lines
for idx, row in launch_sites.iterrows():
    lat, lon = row['lat'], row['long']

    # Add marker for launch site
    folium.Marker([lat, lon], popup=row['launchsite'], icon=folium.Icon(color='blue')).add_to(m)

    # Coastline line
    coast_lat, coast_lon = destination_point(lat, lon, row['Distance_to_Coast_km'], bearings['Coast'])
    folium.PolyLine([(lat, lon), (coast_lat, coast_lon)], color='blue', tooltip='To Coast').add_to(m)

    # Highway line
    highway_lat, highway_lon = destination_point(lat, lon, row['Distance_to_Highway_km'], bearings['Highway'])
    folium.PolyLine([(lat, lon), (highway_lat, highway_lon)], color='green', tooltip='To Highway').add_to(m)

    # City line
    city_lat, city_lon = destination_point(lat, lon, row['Distance_to_City_km'], bearings['City'])
    folium.PolyLine([(lat, lon), (city_lat, city_lon)], color='red', tooltip='To City').add_to(m)

# Show map
m


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Yan Luo](https://www.linkedin.com/in/yan-luo-96288783/)


### Other Contributors


Joseph Santarcangelo


## Change Log


|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|
|2021-05-26|1.0|Yan|Created the initial version|


Copyright © 2021 IBM Corporation. All rights reserved.
